<a href="https://colab.research.google.com/github/AnastasiyaKrokhina/DataCon-time-1/blob/main/notebooks/data1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#satisfying requirements
from pip._internal import main 
packages = ('traitlets', 
            'IPython', 
            'ipywidgets',
            'catboost', 
            'category-encoders',
            'pandas', 
            'numpy', 
            'seaborn', 
            'sklearn',
            'matplotlib',
            'pymatgen',
            'pprint')
for package in packages:
    try:
        __import__(package)
    except ImportError:
        main(['install', package.split()[0]])



import seaborn as sns
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
from pymatgen.ext.matproj import MPRester
from pprint import pprint
MP_API_KEY = "lnFh1UTkg0QO8WxK" #ключ доступа 
m = MPRester(MP_API_KEY)
from pymatgen.core.periodic_table import Element
from pymatgen.core.composition import Composition
from pymatgen.core.structure import Structure, IStructure, Molecule, IMolecule
from pymatgen.core.lattice import Lattice
from pymatgen.core.ion import Ion
from pymatgen.ext.matproj import MPRester
from monty.json import MontyEncoder, MontyDecoder, MSONable


db1=pd.read_excel('https://github.com/AnastasiyaKrokhina/DataCon-time-1/blob/main/data/Database_1.xlsx?raw=true') 
db4=pd.read_excel('https://github.com/AnastasiyaKrokhina/DataCon-time-1/blob/main/data/Database_4.xlsx?raw=true')
db5=pd.read_excel('https://github.com/AnastasiyaKrokhina/DataCon-time-1/blob/main/data/Database_5.xlsx?raw=true')

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached category_encoders-2.5.0-py2.py3-none-any.whl (69 kB)


In [2]:
#db1.columns.values

In [3]:
# # смотрю в каком диапазоне находятся значения
# # значения одинаковые, значит считаю что у данных одна размерность
# db1['Exposure dose (ug/mL)'].describe() 
# db5['dose'].describe()

In [6]:
# переиминовываю дф5 чтобы соеденить с 1 и 4
db5.rename({
    'material': 'Material type',
    'hydro_size': 'Hydro size (nm)',
    'core_size': 'Core size (nm)',
    'surf_charge': 'Surface charge (mV)',
    'surf_area': 'Surface area (m2/g)',
    'dose': 'Exposure dose (ug/mL)', # посмотрела на средние значения для дф1 и подумала, что скорее всего еденицы измерения одинаковые
    'viability': 'Viability (%)',
    'cell_type': 'Cell type'
}, inplace=True, axis=1)

In [7]:
# объедениние баз данных
db1_4 = db1.append(db4, ignore_index=True)
db1_4_5 = db1_4.append(db5, ignore_index=True)
# db1_4_5

In [8]:

db1_4_5.loc[(db1_4_5['Material type'] == "don't remember"), ('Material type')] = ('ZnO')

db1_4_5.loc[(db1_4_5['Elements'] == "Al"), ('Material type')] = ('Al2O3')
db1_4_5.loc[(db1_4_5['Elements'] == "Zn"), ('Material type')] = ('ZnO')

# смотрю сколько всего уникальных решеток 
# видно, что разнообразие небольшое, как и разнообразие материалов

lattice = db1_4_5[['Material type', 
                   #'Elements', # наводка Лины, посмотрела какие элементы у пропусков, заменила наны на элемент оксида
                     'Topological polar surface area (Å²)', 
                     'a (Å)', 'b (Å)', 'c (Å)', 'α (°)', 'β (°)', 'γ (°)', 'Density (g/cm3)']].drop_duplicates(keep='first')
lattice.dropna(inplace=True)
# lattice


In [9]:


for i in lattice.values.tolist():
  # print(i[0])
  # print(lattice2.values.tolist()[1:])
  db1_4_5.loc[(db1_4_5['Material type'] == i[0]), ('Topological polar surface area (Å²)', 'a (Å)', 'b (Å)', 'c (Å)', 'α (°)', 'β (°)', 'γ (°)', 'Density (g/cm3)')] = (i[1:])

# WebCSD https://doi.org/10.1021/acsami.8b05128
# https://www.ccdc.cam.ac.uk/structures/Search?Compound=TiO2&DatabaseToSearch=Published
# https://materialsproject.org/materials/mp-2657/
db1_4_5.loc[(db1_4_5['Material type'] == 'TiO2'), ('Topological polar surface area (Å²)', 
                                                   'a (Å)', 'b (Å)', 'c (Å)', 'α (°)', 'β (°)', 'γ (°)', 'Density (g/cm3)')] = (34.1, 4.664,	4.664,	2.966,	90,	90,	90, 4.13)

# https://materialsproject.org/materials/mp-19306/
db1_4_5.loc[(db1_4_5['Material type'] == 'Fe3O4'), ('Topological polar surface area (Å²)', 
                                                    'a (Å)', 'b (Å)', 'c (Å)', 'α (°)', 'β (°)', 'γ (°)', 'Density (g/cm3)')] = (60.4, 8.244,	8.244,	8.244,	90,	90,	90, 4.95)

# https://materialsproject.org/materials/mp-6930/

db1_4_5.loc[(db1_4_5['Material type'] == 'SiO2'), ('Topological polar surface area (Å²)', 
                                                   'a (Å)', 'b (Å)', 'c (Å)', 'α (°)', 'β (°)', 'γ (°)', 'Density (g/cm3)')] = (34.1, 5.028,	5.028,	5.519,	90,	90,	120, 2.48)
# все остальные тоже с матириалспроджект
db1_4_5.loc[(db1_4_5['Material type'] == 'CuO'), ('Topological polar surface area (Å²)', 
                                                   'a (Å)', 'b (Å)', 'c (Å)', 'α (°)', 'β (°)', 'γ (°)', 'Density (g/cm3)')] = (17.1, 2.993,	2.993,	2.993,	60,	60,	60, 6.97)

db1_4_5.loc[(db1_4_5['Material type'] == 'ZnO'), ('Topological polar surface area (Å²)', 
                                                   'a (Å)', 'b (Å)', 'c (Å)', 'α (°)', 'β (°)', 'γ (°)', 'Density (g/cm3)')] = (17.1, 3.289,	3.289,	5.307,	90,	90,	120, 5.44)

db1_4_5.loc[(db1_4_5['Material type'] == 'Fe2O3'), ('Topological polar surface area (Å²)', 
                                                   'a (Å)', 'b (Å)', 'c (Å)', 'α (°)', 'β (°)', 'γ (°)', 'Density (g/cm3)')] = (43.4, 8.276,	8.276,	8.276,	109,	109,	109, 4.86)

db1_4_5.loc[(db1_4_5['Material type'] == 'In2O3'), ('a (Å)')] = (10.118)

db1_4_5.loc[(db1_4_5['Material type'] == 'Al2O3'), ('Topological polar surface area (Å²)', 
                                                   'a (Å)', 'b (Å)', 'c (Å)', 'α (°)', 'β (°)', 'γ (°)', 'Density (g/cm3)')] = (43.4, 5.178,	5.178,	5.178,	55,	55,	55, 3.87)


In [10]:
# db1_4_5.isna().sum()

In [ ]:
# cell = db1_4_5[['cell_line', 
#                      'Cell type']].drop_duplicates(keep='first')  
# #lattice2.dropna(inplace=True)
# cell

In [ ]:
# db1_4_5['Material type'].unique()

In [11]:
num_atoms = []  
mol_weight = []  #расчет новых дескрипторов
electroneg = []
compositions = []
element_composition = []
anon_formula = []
for column in db1_4_5['Material type']:
  composition = (Composition(column))
  electroneg.append(composition.average_electroneg)

  mol_weight.append(composition.weight)
  num_atoms.append(composition.num_atoms)
  element_composition.append(composition.get_el_amt_dict())
  anon_formula.append(composition.anonymized_formula)
  


In [12]:

for cols in ['Number of atoms', 'Electronegativity', 'Molecular weight (g/mol)']:  #мол вес добавляется дальше, ЭО элемента заменяем на среднюю ЭО вещ-ва, кол-ва атом добавим следом, элементы больше не нужны
  db1_4_5.pop(cols)

In [13]:
db1_4_5['average electronegativity'] = electroneg  #новые дескрипторы -  - в дб1 изначально указана ЭО только для металла, здесь считается средняя ЭО всего вещ-ва


db1_4_5['Molecular weight (g/mol)'] = mol_weight
db1_4_5['Number of atoms'] = num_atoms

In [ ]:
# db1_4_5.isna().sum()

In [ ]:
# db1_4_5.to_csv('db1_4_5_new.csv', header=True) # сохраняем в рабочую папку